In [1]:
!pip install -q keras-tuner
!pip install -q tensorflow
!pip install -q gdown
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt
from google.colab import drive

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 111.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 102.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 116.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.6 MB/s eta 0:00:00


In [2]:
# Mounting Google Drive

drive.mount('/content/drive')

# Loading .npy files

DATA_DIR = '/content/drive/My Drive/EdgeMeter_AI/Data/processed'

X_train = np.load(f'{DATA_DIR}/X_train.npy')
y_train = np.load(f'{DATA_DIR}/y_train.npy')
X_val   = np.load(f'{DATA_DIR}/X_val.npy')
y_val   = np.load(f'{DATA_DIR}/y_val.npy')
X_test  = np.load(f'{DATA_DIR}/X_test.npy')
y_test  = np.load(f'{DATA_DIR}/y_test.npy')

print(f"Train X: {X_train.shape} | Train y: {y_train.shape}")
print(f"Val X:   {X_val.shape} | Val y:   {y_val.shape}")
print(f"Test X:  {X_test.shape} | Test y:  {y_test.shape}")


Mounted at /content/drive
Train X: (10720440, 48) | Train y: (10720440, 12)
Val X:   (1536480, 48) | Val y:   (1536480, 12)
Test X:  (3072960, 48) | Test y:  (3072960, 12)


In [3]:
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.Input(shape=(X_train.shape[1], X_train.shape[2])))
    model.add(layers.LSTM(
        units=hp.Int('units', min_value=32, max_value=128, step=32)
    ))
    model.add(layers.Dropout(
        hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)
    ))
    model.add(layers.Dense(1))

    model.compile(
        optimizer=keras.optimizers.Adam(
            learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
        ),
        loss='mse',
        metrics=['mae']
    )
    return model

In [4]:
print(X_train.shape)


(10720440, 48)


In [5]:
# Reshape
X_train = X_train[..., np.newaxis]
X_val   = X_val[..., np.newaxis]
X_test  = X_test[..., np.newaxis]

print("Input reshaped:", X_train.shape)

Input reshaped: (10720440, 48, 1)


In [6]:
# Creating small tuning subset
subset_size = int(0.01 * X_train.shape[0])  # 1% for tuning

X_train_small = X_train[:subset_size]
y_train_small = y_train[:subset_size]
X_val_small   = X_val[:int(0.01 * X_val.shape[0])]
y_val_small   = y_val[:int(0.01 * y_val.shape[0])]

print(f"Subset: {X_train_small.shape}")

Subset: (107204, 48, 1)


In [7]:
# Setting up Keras Tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,
    executions_per_trial=1,
    directory='tuner_dir',
    project_name='Baseline_LSTM_QuickTune'
)

tuner.search_space_summary()

Search space summary
Default search space size: 3
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
dropout (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [8]:
# Tuning
tuner.search(
    X_train_small, y_train_small,
    epochs=5,
    validation_data=(X_val_small, y_val_small),
    batch_size=32
)

tuner.results_summary()

Trial 5 Complete [00h 04m 40s]
val_loss: 0.019625842571258545

Best val_loss So Far: 0.018933717161417007
Total elapsed time: 00h 36m 12s
Results summary
Results in tuner_dir/Baseline_LSTM_QuickTune
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 1 summary
Hyperparameters:
units: 128
dropout: 0.1
learning_rate: 0.001
Score: 0.018933717161417007

Trial 3 summary
Hyperparameters:
units: 128
dropout: 0.4
learning_rate: 0.001
Score: 0.019329650327563286

Trial 4 summary
Hyperparameters:
units: 64
dropout: 0.1
learning_rate: 0.01
Score: 0.019625842571258545

Trial 2 summary
Hyperparameters:
units: 96
dropout: 0.2
learning_rate: 0.01
Score: 0.019629232585430145

Trial 0 summary
Hyperparameters:
units: 96
dropout: 0.2
learning_rate: 0.0001
Score: 0.01969577744603157


In [9]:
best_hp = tuner.get_best_hyperparameters(1)[0]

print("\nBest hyperparameters:")
print(f"units: {best_hp.get('units')}")
print(f"dropout: {best_hp.get('dropout')}")
print(f"learning_rate: {best_hp.get('learning_rate')}")


✅ Best hyperparameters:
units: 128
dropout: 0.1
learning_rate: 0.001


In [11]:
import json
import os
# Extracting the best HPs as dict
hp_dict = {
    'units': best_hp.get('units'),
    'dropout': best_hp.get('dropout'),
    'learning_rate': best_hp.get('learning_rate')
}

# Saving HPs into jason file to use for model training
hp_path = os.path.join(DATA_DIR, 'best_lstm_hp.json')

with open(hp_path, 'w') as f:
    json.dump(hp_dict, f)

print(f"Saved hyperparameters to: {hp_path}")


Saved hyperparameters to: /content/drive/My Drive/EdgeMeter_AI/Data/processed/best_lstm_hp.json
